# <span style="color:#2C786C">2. 프로젝트: 네이버 영화리뷰 감성분석 도전하기</span>

## <span style="color:#F7B400">목표</span>
> **<span style="color:#2C786C">1. 텍스트 데이터를 머신러닝 입출력용 수치데이터로 변환하는 과정을 이해한다.</span>**<br>
**<span style="color:#2C786C">2. RNN의 특징을 이해하고 시퀀셜한 데이터를 다루는 방법을 이해한다.</span>**<br>
**<span style="color:#2C786C">3. 1-D CNN으로도 텍스트를 처리할 수 있음을 이해한다.</span>**<br>
**<span style="color:#2C786C">4. IMDB와 네이버 영화리뷰 데이터셋을 이용한 영화리뷰 감성분류 실습을 진행한다.</span>**<br>

### <span style="color:#926DD6">참고</span>
>* [딥 러닝을 이용한 자연어 처리 입문(wikidocs) : 네이버 영화 리뷰 감성 분류하기(Naver Movie Review Sentiment Analysis)](https://wikidocs.net/44249)

### <span style="color:#926DD6">용어 정리</span>
**<span style="color:#DD94B9"> [EXPLORATION_DJ 4] 영화리뷰 텍스트 감성분석하기 용어복습</span>**
> ✓ **RNN(Recurrent Neural Network, 순환 신경망)**이란?
>* 인공 신경망의 한 종류
>* 유닛간의 연결이 순환적 구조를 갖는 특징
>* 순방향 신경망과 달리 내부의 메모리를 이용해 시퀀스 형태의 입력을 처리
>* 필기 인식이나 음성 인식과 같이 시변적 특징을 지니는 데이터를 처리하는데 적용
>* 입력받는 신호의 길이가 한정되지 않은 동적 데이터를 처리한다는 점에서 붙여진 이름
>* 추가적인 저장공간을 가질 수 있으며 이 저장공간이 그래프의 형태를 가짐으로써 시간 지연의 기능을 하거나 피드백 루프를 가질 수도 있음
>>* 이를 응용하는 대표적인 예) LSTM과 게이트 순환 유닛(GRU)
>* 구조
>>* 유한 임펄스 구조 : 유향 비순환 그래프(방향 비순환 그래프)
>> ![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Topological_Ordering.svg/220px-Topological_Ordering.svg.png)
>>* 무한 임펄스 구조 : 유향 그래프(방향 그래프)
>> ![](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a2/Directed.svg/220px-Directed.svg.png)

> ✓ **CNN(Convolutional Neural Network, 합성곱 신경망)**이란?
>* 말 그대로 세션 데이터를 기반으로 **유저가 다음에 클릭 또는 구매할 아이템을 예측하는 추천**

> ✓ **감성분석(sentiment analysis)**이란?
>* 말 그대로 세션 데이터를 기반으로 **유저가 다음에 클릭 또는 구매할 아이템을 예측하는 추천**

이전 스텝까지는 영문 텍스트의 감정분석을 진행해 보았습니다. 그렇다면 이번에는 한국어 텍스트의 감정분석을 진행해 보면 어떨까요? 오늘 활용할 데이터셋은 네이버 영화의 댓글을 모아 구성된 [Naver sentiment movie corpus](https://github.com/e9t/nsmc)입니다.

아래와 같이 다운로드를 진행해 주세요.

    $ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
    
    $ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
    
    $ mv ratings_*.txt ~/aiffel/sentiment_classification

프로젝트 진행 순서는 다음과 같습니다.

## <span style="color:#F7B400">순서</span>
> **<span style="color:#2C786C">Step 1. 데이터 준비와 확인</span>**<br>
**<span style="color:#2C786C">Step 2. 데이터로더 구성</span>**<br>
**<span style="color:#2C786C">Step 3. 모델구성을 위한 데이터 분석 및 가공</span>**<br>
**<span style="color:#2C786C">Step 4. 모델구성 및 validation set 구성</span>**<br>
**<span style="color:#2C786C">Step 5. 모델 훈련 개시</span>**<br>
**<span style="color:#2C786C">Step 6. Loss, Accuracy 그래프 시각화</span>**<br>
**<span style="color:#2C786C">Step 7. 학습된 Embedding 레이어 분석</span>**<br>
**<span style="color:#2C786C">Step 8. 한국어 Word2Vec 임베딩 활용하여 성능개선</span>**<br>
**<span style="color:#2C786C">Step 9. 루브릭 평가</span>**<br>
**<span style="color:#2C786C">Step 10. 회고</span>**

## <span style="color:#124445">Step 1. 데이터 준비와 확인</span>
---

In [4]:
import pandas as pd
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import re

from konlpy.tag import Mecab
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

%matplotlib inline

# 데이터를 읽어봅시다. 
train_data = pd.read_table('~/aiffel/sentiment_classification/ratings_train.txt')
test_data = pd.read_table('~/aiffel/sentiment_classification/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## <span style="color:#124445">Step 2. 데이터로더 구성</span>
---
실습때 다루었던 IMDB 데이터셋은 텍스트를 가공하여 imdb.data_loader() 메소드를 호출하면 숫자 인덱스로 변환된 텍스트와 word_to_index 딕셔너리까지 친절하게 제공합니다. 그러나 이번에 다루게 될 nsmc 데이터셋은 전혀 가공되지 않은 텍스트 파일로 이루어져 있습니다. 이것을 읽어서 imdb.data_loader()와 동일하게 동작하는 자신만의 **`data_loader`**를 만들어 보는 것으로 시작합니다. **`data_loader`** 안에서는 다음을 수행해야 합니다.

>* 데이터의 중복 제거
>* NaN 결측치 제거
>* 한국어 토크나이저로 토큰화
>* 불용어(Stopwords) 제거
>* 사전word_to_index 구성
>* 텍스트 스트링을 사전 인덱스 스트링으로 변환
>* X_train, y_train, X_test, y_test, word_to_index 리턴

In [5]:
# from konlpy.tag import Mecab

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    train_data.drop_duplicates(subset=['document'], inplace=True)
    train_data = train_data.dropna(how = 'any') 
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 

    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_train.append(temp_X)

    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) # 토큰화
        temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        X_test.append(temp_X)

    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['<PAD>', '<BOS>', '<UNK>', '<UNUSED>'] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}

    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in wordlist]

    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))

    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data)

In [6]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [7]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

## <span style="color:#124445">Step 3. 모델구성을 위한 데이터 분석 및 가공/span>
---
* 데이터셋 내 문장 길이 분포
* 적절한 최대 문장 길이 지정
* keras.preprocessing.sequence.pad_sequences 을 활용한 패딩 추가

## <span style="color:#124445">Step 4. 모델구성 및 validation set 구성</span>
---
모델은 3가지 이상 다양하게 구성하여 실험해 보세요.

## <span style="color:#124445">Step 5. 모델 훈련 개시</span>
---

## <span style="color:#124445">Step 6. Loss, Accuracy 그래프 시각화</span>
---

## <span style="color:#124445">Step 7. 학습된 Embedding 레이어 분석</span>
---

## <span style="color:#124445">Step 8. 한국어 Word2Vec 임베딩 활용하여 성능개선</span>
---
* [Pre-trained word vectors of 30+ languages](https://github.com/Kyubyong/wordvectors)

## <span style="color:#124445">Step 9. 루브릭 평가</span>
---

|평가문항|상세기준|
|-------|--------|
|1. 다양한 방법으로 Text Classification 태스크를 성공적으로 구현하였다.|3가지 이상의 모델이 성공적으로 시도됨|
|gensim을 활용하여 자체학습된 혹은 사전학습된 임베딩 레이어를 분석하였다.|gensim의 유사단어 찾기를 활용하여 자체학습한 임베딩과 사전학습 임베딩을 적절히 분석함|
|3. 한국어 Word2Vec을 활용하여 가시적인 성능향상을 달성했다.|네이버 영화리뷰 데이터 감성분석 정확도를 85% 이상 달성함|

## <span style="color:#2C786C">Step 10. 회고</span>
---
첫 번째 NLP 과제인데 아직 코드가 낯선 제게는 너무 어려웠습니다.😖<br>
코드가 실행 되기 위해 미리 설치해야하는 패키지를 안 알려줘서 더욱 힘들었습니다.<br>
퍼실님이 알려주셨지만 저 같은 노베이스를 위해 더더더더더 노드가 친절하면 좋겠어요.<br>
3번 카메라 과제는 컴퓨터 교환 후 셋팅하느라 고생했는데 영화리뷰도 만만치 않네요.<br>
같은 조원분들도 어렵다고 하셔서 겁이 났지만 그래도 달성하니 뿌듯하네요.<br>
참고자료들을 찾아 복습해야겠습니다.😭

---